In [9]:
import requests
import json
from pathlib import Path
import sys

# Define Overpass API endpoint and the Query
overpass_url = "https://overpass-api.de/api/interpreter"

# --- STABLE QUERY (matching the working nightclub script) ---
# We are reverting to the [out:json] format and the complex
# 'out body' footer, as this is proven to work in your other script.
overpass_query = """
[out:json][timeout:180];

/* 1. Get the Berlin boundary by its Area ID */
area(3600062422)->.berlin;

/* 2. Find all objects (explicitly) with the desired tags */
(
  node["amenity"="doctors"](area.berlin);
  way["amenity"="doctors"](area.berlin);
  relation["amenity"="doctors"](area.berlin);

  node["amenity"="clinic"](area.berlin);
  way["amenity"="clinic"](area.berlin);
  relation["amenity"="clinic"](area.berlin);
);

/* 3. Output the results (the classic, stable way) */
out body;
>;
out skel qt;
"""

# --- (Rest of the script is unchanged) ---

# Define the output file path
output_dir = Path('../source')
output_dir.mkdir(parents=True, exist_ok=True) # Ensure 'source' folder exists
output_geojson_file = output_dir / 'doctors_and_clinics_raw.geojson' 

# Send the request to Overpass API
print("Sending query to Overpass API for doctors and clinics...") 
print("(this may take up to 180 seconds)")
try:
    response = requests.post(overpass_url, data=overpass_query)
    
    # Check if the request was successful
    response.raise_for_status() 
    print("Query successful. Data received.")
    
    # Get the JSON data from the response
    data = response.json()

    # Save the raw JSON data to a .geojson file
    with open(output_geojson_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"✅ Raw JSON data successfully saved to: {output_geojson_file}")

except requests.exceptions.RequestException as e:
    print(f"❌ An error occurred connecting to Overpass API: {e}")
    # Print server error message if available
    if e.response:
        print(f"Server response:\n{e.response.text}")
    sys.exit(1) # Exit the script if the download failed

except json.JSONDecodeError:
    print("❌ Failed to decode JSON from the response. The API might be down or the query is wrong.")
    print(f"Server response:\n{response.text}")
    sys.exit(1)

except Exception as e:
    print(f"An unexpected error occurred: {e}")
    sys.exit(1)

Sending query to Overpass API for doctors and clinics...
(this may take up to 180 seconds)
Query successful. Data received.
✅ Raw JSON data successfully saved to: ..\source\doctors_and_clinics_raw.geojson
